In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
teams = pd.read_csv("/Users/shravann/Downloads/Portfolio db/linear reg. project/teams.csv")

In [4]:
train, test = train_test_split(teams, test_size=0.2, random_state=1)

In [5]:
predictors = ["athletes", "events"]
target = "medals"

In [6]:
X = train[predictors].copy()
y = train[[target]].copy()

In [7]:
X

,athletes,events
1322,6,6
1872,119,80
953,4,4
1117,2,2
1993,43,25
...,...,...
1791,40,25
1096,36,23
1932,719,245
235,13,11


In [8]:
y

,medals
1322,0
1872,5
953,0
1117,0
1993,0
...,...
1791,1
1096,1
1932,264
235,0


In [9]:
x_mean = X.mean()
x_std = X.std()

In [10]:
X = (X - x_mean) / x_std

In [11]:
X["intercept"] = 1

In [12]:
X = X[["intercept"] + predictors]

In [13]:
X.describe()

,intercept,athletes,events
count,1611.0,1.611000e+03,1.611000e+03
mean,1.0,-8.269818e-18,-9.923781e-18
std,0.0,1.000000e+00,1.000000e+00
min,1.0,-5.768883e-01,-7.143930e-01
25%,1.0,-5.297371e-01,-6.123079e-01
50%,1.0,-4.197174e-01,-4.489717e-01
75%,1.0,-2.679027e-02,1.839560e-01
max,1.0,6.008571e+00,4.634867e+00


In [14]:
X.T

,1322,1872,953,1117,1993,385,1287,1831,0,1159,...,960,847,1669,715,905,1791,1096,1932,235,1061
intercept,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
athletes,-0.537596,0.350420,-0.553313,-0.569030,-0.246829,-0.482586,-0.537596,0.138239,-0.521879,-0.152527,...,-0.199678,-0.160386,-0.529737,-0.529737,-0.341132,-0.270405,-0.301839,5.065546,-0.482586,-0.191820
events,-0.612308,0.898552,-0.653142,-0.693976,-0.224384,-0.571474,-0.612308,0.102288,-0.571474,-0.163133,...,-0.285636,-0.101882,-0.612308,-0.591891,-0.367304,-0.224384,-0.265219,4.267361,-0.510223,0.041037


In [15]:
alpha = 2
I = np.identity(X.shape[1])
penalty = alpha * I

In [16]:
penalty

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [17]:
penalty[0][0] = 0

In [18]:
penalty

array([[0., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [19]:
B = np.linalg.inv(X.T @ X + penalty) @ X.T @ y

In [20]:
B

,medals
0,10.691496
1,61.857734
2,-34.632920


In [21]:
B.index = ["intercept", "athletes", "events"]

In [22]:
test_X = test[predictors]
test_X = (test_X - x_mean) / x_std
test_X["intercept"] = 1
test_X = test_X[["intercept"] + predictors]
    
predictions = test_X @ B

In [23]:
test_X

,intercept,athletes,events
309,1,-0.553313,-0.653142
285,1,0.594035,1.000637
919,1,-0.144668,0.102288
120,1,0.146098,0.531045
585,1,-0.301839,-0.122299
...,...,...,...
541,1,-0.380425,-0.408138
1863,1,-0.191820,0.143122
622,1,-0.058224,0.388126
1070,1,-0.569030,-0.693976


In [24]:
predictions

,medals
309,-0.914959
285,12.782156
919,-1.799893
120,1.337116
585,-3.744014
...,...
541,1.294285
1863,-6.130765
622,-6.352080
1070,-0.472980


In [25]:
def ridge_fit(train, predictors, target, alpha):
    X = train[predictors].copy()
    y = train[[target]].copy()
    
    x_mean = X.mean()
    x_std = X.std()
    
    X = (X - x_mean) / x_std
    X["intercept"] = 1
    X = X[["intercept"] + predictors]
    
    penalty = alpha * np.identity(X.shape[1])
    penalty[0][0] = 0
    
    B = np.linalg.inv(X.T @ X + penalty) @ X.T @ y
    B.index = ["intercept", "athletes", "events"]
    return B, x_mean, x_std

In [26]:
B, x_mean, x_std = ridge_fit(train, predictors, target, alpha)

In [27]:
def ridge_predict(test, predictors, x_mean, x_std, B):
    test_X = test[predictors]
    test_X = (test_X - x_mean) / x_std
    test_X["intercept"] = 1
    test_X = test_X[["intercept"] + predictors]

    predictions = test_X @ B
    return predictions

In [28]:
predictions = ridge_predict(test, predictors, x_mean, x_std, B)

In [29]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=alpha)

In [30]:
ridge.fit(X[predictors], y)

Ridge(alpha=2)

In [31]:
ridge.coef_

array([[ 61.85773366, -34.63292036]])

In [32]:
ridge.intercept_

array([10.69149597])

In [33]:
sklearn_predictions = ridge.predict(test_X[predictors])

In [34]:
predictions - sklearn_predictions

,medals
309,-1.865175e-14
285,-1.882938e-13
919,-1.791900e-13
120,-2.309264e-13
585,-1.567635e-13
...,...
541,-3.952394e-14
1863,-2.398082e-13
622,-2.930989e-13
1070,7.771561e-16


In [35]:
from sklearn.metrics import mean_absolute_error

errors = []
alphas = [10**i for i in range(-2,4)]

for alpha in alphas:
    B, x_mean, x_std = ridge_fit(train, predictors, target, alpha)
    predictions = ridge_predict(test, predictors, x_mean, x_std, B)
    
    errors.append(mean_absolute_error(test[target], predictions))

In [36]:
errors

[6.309640830161143,
 6.306044331952908,
 6.27228337643161,
 6.114051204717738,
 7.156811236590453,
 6.9780545895757315]

In [37]:
alphas


[0.01, 0.1, 1, 10, 100, 1000]